# Мазмұнға негізделген сүзгілеу үшін терең үйрену

Осы жаттығуда, сіз нейрондық желіні пайдаланып, фильмдерге арналған ұсыныс жүйесін құру үшін мазмұнға негізделген сүзгілеуді жүзеге асырасыз. Сіздің тапсырмаңыз соңында беріледі.

Біз таныс пакеттерді, яғни NumPy, TensorFlow және [scikit-learn](https://scikit-learn.org/stable/)-нің пайдалы функцияларын қолданамыз. Сондай-ақ, кестелерді әдемі түрде шығару үшін [tabulate](https://pypi.org/project/tabulate/) және кестелік деректерді ұйымдастыру үшін [Pandas](https://pandas.pydata.org/) пайдаланамыз.

In [ ]:
import numpy as np
import numpy.ma as ma
from numpy import genfromtxt
from collections import defaultdict
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import tabulate
from recsysNN_utils import *
pd.set_option("display.precision", 1)
from IPython.display import HTML

## Фильм бағалау деректері  
Деректер жиынтығы [MovieLens ml-latest-small](https://grouplens.org/datasets/movielens/latest/) жинағынан алынған.

Бастапқы деректер жиынтығында 600 пайдаланушы тарапынан бағаланған 9000 фильм бар, бағалау 0.5-тен 5-ке дейінгі шкала бойынша, 0.5 қадамдармен берілген. 2000 жылдан бастап шыққан және танымал жанрлардағы фильмдерге назар аудару мақсатында, деректер жиынтығы кішірейтілді. Кішірейтілген жиынтықта 395 пайдаланушы және 694 фильм бар. Әр фильм үшін жиынтықта фильм атауы, шыққан жылы және бір немесе бірнеше жанр беріледі. Мысалы, *Toy Story 3* фильмі 2010 жылы шыққан және бірнеше жанрға ие: "Adventure|Animation|Children|Comedy|Fantasy|IMAX". Бұл жиынтық пайдаланушылар туралы бағалауларынан басқа өте аз ақпарат қамтиды және төменде сипатталған нейрондық желілерді оқыту векторларын жасау үшін қолданылады.

### Нейрондық желімен мазмұнға негізделген сүзгілеу

Бірлескен сүзгілеу зертханасында, сіз пайдаланушы векторын және фильм/зат векторын генерацияладыңыз, олардың нүктелік көбейтіндісі бағаны болжау үшін қолданылды. Бұл векторлар тек бағалаулардан алынған болатын.

Мазмұнға негізделген сүзгілеу де пайдаланушы және фильмнің ерекшелік векторын жасайды, бірақ пайдаланушы мен/немесе фильм туралы болжауды жақсартатын қосымша ақпараттың да болуы мүмкін екенін мойындайды. Қосымша ақпарат нейрондық желіге беріледі, ол пайдаланушы мен фильм векторын генерациялайды.

Желідегі фильм контенті бастапқы деректер мен кейбір «инженерлік ерекшеліктердің» комбинациясынан тұрады. Бастапқы ерекшеліктерге фильмнің шыққан жылы және бір ыстық вектор түрінде ұсынылған фильмнің жанры кіреді. 14 жанр бар. Инженерлік ерекшелік – пайдаланушы бағаларынан алынған орташа баға. Бірнеше жанры бар фильмдер әр жанр үшін оқыту векторын алады.

Пайдаланушы контенті тек инженерлік ерекшеліктерден құралады. Әр пайдаланушы үшін жанр бойынша орташа баға есептеледі. Сонымен қатар, пайдаланушы идентификаторы, баға саны және орташа баға бар, бірақ олар оқыту немесе болжау контентіне қосылмайды, олар деректерді түсіндіруде пайдалы.

Оқыту жиынтығы деректер жиынтығындағы барлық пайдаланушы бағаларынан тұрады. Пайдаланушы және фильм/зат векторлары жоғарыда көрсетілген желіге оқыту жиынтығы ретінде бірге беріледі. Пайдаланушы векторы пайдаланушы берген барлық фильмдер үшін бірдей.

Төменде, кейбір деректерді жүктеп, көрсетейік.


In [ ]:
# Деректерді жүктеу, баптау айнымалыларын орнату
item_train, user_train, y_train, item_features, user_features, item_vecs, movie_dict, user_to_genre = load_data()

num_user_features = user_train.shape[1] - 3  # оқыту кезінде пайдаланушы идентификаторын, бағалау санын және орташа бағалауды алып тастау
num_item_features = item_train.shape[1] - 1  # оқыту кезінде фильм идентификаторын алып тастау
uvs = 3  # пайдаланушы жанр векторының басталу индексі
ivs = 3  # фильм жанр векторының басталу индексі
u_s = 3  # оқытуда пайдаланылатын бағаналардың басталу индексі, пайдаланушы
i_s = 1  # оқытуда пайдаланылатын бағаналардың басталу индексі, элементтер
scaledata = True  # деректерге стандартты масштабтау қолданылатынын белгілейді, егер шын болса
print(f"Оқыту векторларының саны: {len(item_train)}")


Пайдаланушы және элемент/фильм ерекшеліктерінің кейбіреулері оқыту кезінде пайдаланылмайды. Төменде жақшадағы "[]" ішіндегі, мысалы, "пайдаланушы идентификаторы", "бағалау саны" және "орташа баға" сияқты ерекшеліктер модель оқытылып қолданылғанда қосылмайды. Назар аударыңыз, барлық бағаланған фильмдер үшін пайдаланушы векторы бірдей.

In [ ]:
HTML(pprint_train(user_train, user_features, uvs,  u_s, maxcount=5))

In [ ]:
HTML(pprint_train(item_train, item_features, ivs, i_s, maxcount=5, user=False))

In [ ]:
print(f"y_train[:5]: {y_train[:5]}")

Жоғарыда 6874 нөмірлі фильмнің 2003 жылы шыққан экшн фильмі екені көрініп тұр. Пайдаланушы 2 экшн фильмдерді орта есеппен 3.9 бағалайды. Сонымен қатар, 6874 нөмірлі фильм қылмыс және триллер жанрларында да тіркелген. MovieLens пайдаланушылары бұл фильмге орташа 4 баға берген. Оқыту үлгісі екі кестеден бір жол және y_train бағанындағы бағадан тұрады.

In [ ]:
# масштабты оқыту деректері
if scaledata:
    item_train_save = item_train
    user_train_save = user_train

    scalerItem = StandardScaler()
    scalerItem.fit(item_train)
    item_train = scalerItem.transform(item_train)

    scalerUser = StandardScaler()
    scalerUser.fit(user_train)
    user_train = scalerUser.transform(user_train)

    print(np.allclose(item_train_save, scalerItem.inverse_transform(item_train)))
    print(np.allclose(user_train_save, scalerUser.inverse_transform(user_train)))

Нәтижелерді бағалау үшін, деректерді Course 2, апта 3-тен талқыланғандай оқыту және тест жиынтықтарына бөлеміз. Мұнда деректерді бөлу және араластыру үшін [sklearn train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) қолданамыз. Бастапқы кездейсоқ күйді бірдей мәнге орнату item, user және y элементтерінің бірдей түрде араластырылуын қамтамасыз етеді.

In [ ]:
item_train, item_test = train_test_split(item_train, train_size=0.80, shuffle=True, random_state=1)
user_train, user_test = train_test_split(user_train, train_size=0.80, shuffle=True, random_state=1)
y_train, y_test       = train_test_split(y_train,    train_size=0.80, shuffle=True, random_state=1)
print(f"фильм/зат оқыту деректерінің өлшемі: {item_train.shape}")
print(f"фильм/зат тест деректерінің өлшемі: {item_test.shape}")


Масштабталған, араластырылған деректердің орташа мәні енді нөлге тең.

In [ ]:
HTML(pprint_train(user_train, user_features, uvs, u_s, maxcount=5))

Мақсатты бағалауларды Min Max Scaler арқылы -1 мен 1 аралығында масштабтаңыз. Біз scikit-learn қолданамыз, өйткені оның inverse_transform функциясы бар. [scikit learn MinMaxScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html)

In [ ]:
scaler = MinMaxScaler((-1, 1))
scaler.fit(y_train.reshape(-1, 1))
ynorm_train = scaler.transform(y_train.reshape(-1, 1))
ynorm_test = scaler.transform(y_test.reshape(-1, 1))
print(ynorm_train.shape, ynorm_test.shape)

## Мазмұнға негізделген сүзгілеу үшін нейрондық желі

Енді жоғарыда көрсетілген суретте сипатталғандай нейрондық желіні құрамыз. Ол екі желіден тұрады, олар нүктелік көбейтінді арқылы біріктіріледі. Сіз осы екі желіні құрастырасыз. Бұл мысалда олар бірдей болады. Алайда, осы желілер бірдей болу қажет емес. Егер пайдаланушы контенті фильм контентінен едәуір үлкен болса, сіз пайдаланушы желісінің күрделілігін фильм желісіне қарағанда арттыруды таңдай аласыз. Бұл жағдайда контент ұқсас болғандықтан, желілер бірдей.

- Keras Sequential моделін қолданыңыз:
    - Бірінші қабат – 256 бірлікті және relu активациясын қолданатын тығыз қабат.
    - Екінші қабат – 128 бірлікті және relu активациясын қолданатын тығыз қабат.
    - Үшінші қабат – `num_outputs` бірлікті және сызықтық немесе активациясыз тығыз қабат.
    
Желі қалғасын кодпен қамтамасыз етіледі. Қамтамасыз етілген код Keras Sequential моделін емес, Keras [Functional API](https://keras.io/guides/functional_api/) моделін қолданады. Бұл формат компоненттердің өзара байланысын икемді түрде құруға мүмкіндік береді.

In [ ]:
num_outputs = 32
tf.random.set_seed(1)
user_NN = tf.keras.models.Sequential([
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_outputs, activation='linear'),
])

item_NN = tf.keras.models.Sequential([
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_outputs, activation='linear'),
])

# пайдаланушы енгізімін жасап, негізгі желіге бағыттау
input_user = tf.keras.layers.Input(shape=(num_user_features,))
vu = user_NN(input_user)
vu = tf.keras.layers.Lambda(lambda x: tf.linalg.l2_normalize(x, axis=1))(vu)

# элемент енгізімін жасап, негізгі желіге бағыттау
input_item = tf.keras.layers.Input(shape=(num_item_features,))
vm = item_NN(input_item)
vm = tf.keras.layers.Lambda(lambda x: tf.linalg.l2_normalize(x, axis=1))(vm)

# vu және vm екі векторының нүктелік көбейтіндісін есептеу
output = tf.keras.layers.Dot(axes=1)([vu, vm])

# модельдің енгізімдері мен шығуын анықтау
model = Model([input_user, input_item], output)

model.summary()

Біз орташа квадраттық қате шығын функциясын және Adam оптимизаторын қолданамыз.

In [ ]:
tf.random.set_seed(1)
cost_fn = tf.keras.losses.MeanSquaredError()
opt = keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=opt,
              loss=cost_fn)

In [ ]:
tf.random.set_seed(1)
model.fit([user_train[:, u_s:], item_train[:, i_s:]], ynorm_train, epochs=30)

Модельді тест деректері бойынша жоғалтуды анықтау үшін бағалаңыз. Ол оқыту жоғалтумен салыстырмалы, яғни модель оқыту деректеріне айтарлықтай артық бейімделмегенін көрсетеді.

In [ ]:
model.evaluate([user_test[:, u_s:], item_test[:, i_s:]], ynorm_test)

### 3.1 Болжаулар
Төменде сіз моделі арқылы әртүрлі жағдайларда болжаулар жасайсыз.

#### Жаңа пайдаланушыға арналған болжаулар
Алдымен, біз жаңа пайдаланушы жасап, сол пайдаланушыға фильмдерді ұсынуды модельден сұраймыз. Мысал пайдаланушы контентін қолданып көргеннен кейін, өз қалауыңызға сәйкес пайдаланушы контентін өзгертіп, модельдің не ұсынатынын тексеруге болады. Ескеріңіз, бағалар 0.5-тен 5.0-ға дейінгі диапазонда, жарты қадамдық ұлғайту арқылы беріледі.

In [ ]:
new_user_id = 5000
new_rating_ave = 1.0
new_action = 1.0
new_adventure = 1
new_animation = 1
new_childrens = 1
new_comedy = 5
new_crime = 1
new_documentary = 1
new_drama = 1
new_fantasy = 1
new_horror = 1
new_mystery = 1
new_romance = 5
new_scifi = 5
new_thriller = 1
new_rating_count = 3

user_vec = np.array([[new_user_id, new_rating_count, new_rating_ave,
                      new_action, new_adventure, new_animation, new_childrens,
                      new_comedy, new_crime, new_documentary,
                      new_drama, new_fantasy, new_horror, new_mystery,
                      new_romance, new_scifi, new_thriller]])

Жаңа пайдаланушы үшін ең жоғары бағаланған фильмдерді қарастырайық. Ескеріңіз, пайдаланушы векторы комедия және романтикалық жанрларға басымдық берген болатын.  
Төменде оқыту/тест жиынтығындағы әр фильмге арналған векторлары бар `item_vecs` жиынтығын қолданамыз. Бұл векторлар жоғарыда берілген пайдаланушы векторымен сәйкестендіріліп, масштабталған векторлар жаңа пайдаланушыға арналған барлық фильмдерге бағаларды болжау үшін пайдаланылады.

In [ ]:
# деректер жиынындағы фильмдер санына сәйкес пайдаланушы векторын жасап, қайталау.
user_vecs = gen_user_vecs(user_vec, len(item_vecs))

# векторларды масштабтап, барлық фильмдер үшін болжау жасау. Нәтижелерді баға бойынша сұрыпталған түрде қайтарады.
sorted_index, sorted_ypu, sorted_items, sorted_user = predict_uservec(user_vecs, item_vecs, model, u_s, i_s,
                                                                       scaler, scalerUser, scalerItem, scaledata=scaledata)

HTML(print_pred_movies(sorted_ypu, sorted_user, sorted_items, movie_dict, maxcount=10))

Егер жоғарыда пайдаланушы векторын құрсаңыз, желі пайдаланушы жанрларының **жиынтығын** қамтитын вектор негізінде бағаны болжауға үйретілгенін ескеру маңызды. Тек бір жанрға ең жоғары баға беріп, қалған жанрларға ең төмен баға беру желі үшін мағыналы болмауы мүмкін, егер оқыту деректерінде мұндай баға жиынтығы бар пайдаланушылар болмаған болса.

#### Бар пайдаланушы үшін болжаулар  
"Пайдаланушы 36" — деректер жиынындағы пайдаланушылардың бірі — үшін болжауларға назар аударайық. Біз болжауларды модель бағаларымен салыстыра аламыз. Назар аударыңыз, бірнеше жанрға жататын фильмдер оқыту деректерінде бірнеше рет кездеседі. Мысалы, *The Time Machine* фильмі үш жанрға жатады: **Adventure**, **Action**, **Sci-Fi**.

In [ ]:
uid = 36
# пайдаланушы векторларының жиынын қалыптастыру. Бұл бірдей вектор, түрлендірілген және қайталанған.
user_vecs, y_vecs = get_user_vecs(uid, scalerUser.inverse_transform(user_train), item_vecs, user_to_genre)

# векторларды масштабтап, барлық фильмдер үшін болжау жасау. Нәтижелерді баға бойынша сұрыпталған түрде қайтарады.
sorted_index, sorted_ypu, sorted_items, sorted_user = predict_uservec(user_vecs, item_vecs, model, u_s, i_s, scaler,
                                                                      scalerUser, scalerItem, scaledata=scaledata)
sorted_y = y_vecs[sorted_index]

# сұрыпталған болжауларды басып шығару
HTML(print_existing_user(sorted_ypu, sorted_y.reshape(-1,1), sorted_user, sorted_items, item_features, ivs, uvs, movie_dict, maxcount=10))


#### Ұқсас элементтерді табу  
Жоғарыдағы нейрондық желі екі ерекшелік векторын шығарады: пайдаланушы ерекшелік векторы $v_u$ және фильм ерекшелік векторы $v_m$. Бұл — мағынасын түсіндіру қиын 32 өлшемді векторлар. Алайда, ұқсас элементтердің векторлары да ұқсас болады. Бұл ақпаратты ұсыным жасау үшін пайдалануға болады. Мысалы, егер пайдаланушы "Toy Story 3" фильміне жоғары баға берген болса, ұқсас фильм ерекшелік векторлары бар басқа фильмдерді ұсынып көруге болады.

Ұқсастықты өлшеудің бір тәсілі — екі вектордың $ \mathbf{v_m^{(k)}}$ және $\mathbf{v_m^{(i)}}$ арасындағы квадраттық қашықтықты есептеу:
$$\left\Vert \mathbf{v_m^{(k)}} - \mathbf{v_m^{(i)}}  \right\Vert^2 = \sum_{l=1}^{n}(v_{m_l}^{(k)} - v_{m_l}^{(i)})^2\tag{1}$$

### Квадраттық қашықтықты есептейтін функция.

In [ ]:
def sq_dist(a, b):
    """
    Екі вектор арасындағы квадраттық қашықтықты қайтарады.

    Параметрлер:
      a (ndarray (n,)): n ерекшелігі бар вектор
      b (ndarray (n,)): n ерекшелігі бар вектор

    Қайтарымы:
      d (float): қашықтық
    """
    d = sum(np.square(a - b))
    return d

In [ ]:
a1 = np.array([1.0, 2.0, 3.0]); b1 = np.array([1.0, 2.0, 3.0])
a2 = np.array([1.1, 2.1, 3.1]); b2 = np.array([1.0, 2.0, 3.0])
a3 = np.array([0, 1, 0]);       b3 = np.array([1, 0, 0])

print(f"a1 мен b1 арасындағы квадраттық қашықтық: {sq_dist(a1, b1)}")
print(f"a2 мен b2 арасындағы квадраттық қашықтық: {sq_dist(a2, b2)}")
print(f"a3 мен b3 арасындағы квадраттық қашықтық: {sq_dist(a3, b3)}")


Фильмдер арасындағы қашықтықтар матрицасын модель бір рет оқытылғаннан кейін есептеп, оны қайта оқытпай-ақ жаңа ұсынымдар үшін қайта пайдалануға болады. Модель оқытылып болғаннан кейінгі алғашқы қадам — әр фильм үшін фильм ерекшелік векторын, яғни $v_m$ алуға кірісу. Бұл үшін біз оқытылған `item_NN` желісін пайдаланамыз және фильм векторларын өткізіп, $v_m$ генерациялау үшін шағын қосымша модель құрамыз.

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Lambda

input_item_m = Input(shape=(num_item_features,))     # енгізу қабаты
vm_m = item_NN(input_item_m)                         # оқытылған item_NN қолдану
vm_m = Lambda(lambda x: tf.linalg.l2_normalize(x, axis=1))(vm_m)  # қалыпқа келтіруді Lambda қабатымен орындау
model_m = Model(input_item_m, vm_m)                  # модель құру
model_m.summary()                                    # модель құрылымын шығару


Фильм моделін алғаннан кейін, фильм ерекшелік векторларының жиынын құруға болады — бұл үшін модельге `item_vecs` сияқты фильм/элемент векторларының жиынын енгізу арқылы болжау жүргізіледі. `item_vecs` — барлық фильм векторларының жиыны. Есіңізде болсын, бір фильм әр жанр үшін жеке вектор ретінде кездеседі. Бұл векторларды оқытылған модельмен қолдану үшін алдын ала масштабтау қажет. Болжау нәтижесінде әр фильм үшін 32 өлшемді ерекшелік векторы алынады.

In [ ]:
scaled_item_vecs = scalerItem.transform(item_vecs)
vms = model_m.predict(scaled_item_vecs[:, i_s:])
print(f"Болжанған барлық фильм ерекшелік векторларының өлшемі: {vms.shape}")

Енді әр фильм ерекшелік векторы мен қалған барлық фильм ерекшелік векторлары арасындағы квадраттық қашықтық матрицасын есептейік.

Содан кейін әр жол бойынша минимумды табу арқылы ең жақын фильмді анықтай аламыз. Бұл жерде біз [numpy masked arrays](https://numpy.org/doc/1.21/user/tutorial-ma.html) — маскаланған массивтерді қолданамыз, себебі олар бір фильмнің өзін таңдап алуын болдырмайды. Диагональ бойындағы маскаланған мәндер есептеуге қосылмайды.

# 1-тапсырма: Пайдаланушыға жекелендірілген фильм ұсынымдарын жасау функциясы

## **Функция не істеуі керек:**

**Input:**
- Пайдаланушының ID-сі (мысалы, `uid = 36`)
- Пайдаланушы туралы деректер (мысалы, оның жанрларға деген қалауы немесе бағалау тарихы)
- Фильм/элемент деректері және алдын ала оқытылған модель

**Өңдеу:**
- Берілген пайдаланушы ID-сі бойынша пайдаланушының ерекшелік векторын табу немесе жасау
- Осы векторды деректер жиынындағы әр фильм үшін қайталау — әр фильмге жеке баға болжау үшін
- Енгізілген векторларды модель талап ететін форматқа сәйкестендіріп масштабтау
- Оқытылған модельді қолданып, пайдаланушыға әр фильм қаншалықты ұнайтынын болжау
- Фильмдерді болжау нәтижелері бойынша сұрыптау (ең жоғарғы ұсынылғаннан төменге қарай)

**output:**
- Осы пайдаланушы үшін ұсынылатын ең үздік фильмдердің тізімін (немесе кестесін) шығару
- Төмендегідей мәліметтерді көрсету:
  - Фильм атауы  
  - Жанры  
  - Болжау бойынша баға  
  - (Қосымша) Нақты баға, егер қолжетімді болса

---

**Мысал:**

36-нөмірлі пайдаланушы көбіне экшн және оқиғаға толы фильмдерді көреді. Функцияны орындағанда төмендегідей нәтиже берілуі мүмкін:

| Фильм атауы            | Жанр      | Болжау бағасы |
|------------------------|-----------|----------------|
| Avengers: Endgame      | Action    | 4.9            |
| Mad Max: Fury Road     | Adventure | 4.8            |
| John Wick              | Action    | 4.7            |


### 2-Тапсырма: Фильмге ұқсас 10 фильмді табу

Функция жазу — оған фильмнің атауы беріледі, ал нәтиже ретінде **ең ұқсас 10 фильмнің** тізімін қайтарады (векторлық ұқсастыққа негізделген).

---

### Функция талаптары

- **Input:**  
  - `movie_name` (*str*): Ізделетін фильмнің атауы.
  - Қосымша параметрлер: фильм векторлары, метадеректер, ұқсастық пен жанрды есептейтін көмекші функциялар.

- **Процесс:**  
  1. Фильмнің атауы арқылы оның векторлық ұсынуын табу.
  2. Осы фильм мен қалған барлық фильмдердің арасындағы ұқсастықты (мысалы, евклидтік қашықтық) есептеу.
  3. Ұқсастық дәрежесіне қарай фильмдерді сұрыптап, ең ұқсас 10 фильмді таңдау (бастапқы фильмді қоспағанда).
  4. Бастапқы фильм мен табылған фильмдердің атаулары мен жанрларын шығару.

- **Output:**  
  - Кесте немесе тізім, мына ақпаратпен:
    - Енгізілген фильмнің атауы және жанры.
    - Ең ұқсас 10 фильмнің атаулары мен жанрлары.

---

### Мысал

| Бастапқы фильм | Жанры     | Ұқсас фильм     | Жанры     |
|----------------|-----------|------------------|-----------|
| Inception      | Ғылыми-фантастика | Interstellar   | Ғылыми-фантастика |
| Inception      | Ғылыми-фантастика | The Matrix     | Экшн      |
| ...            | ...       | ...              | ...       |

